## ConfigBox - Used for error handling


In [1]:
import yaml

yaml_file_path = "test.yaml"

# Open the YAML file and load its contents
with open(yaml_file_path, 'r') as yaml_file:
    data = yaml.safe_load(yaml_file)


print(data)
print(type(data))

{'name': 'Rajjo', 'email': 'relkarrajat@gmail.com'}
<class 'dict'>


In [5]:
data['name']

'Rajjo'

In [7]:
from box import ConfigBox

config_data = ConfigBox(data)

print(config_data.name)
print(config_data.email)

Rajjo
relkarrajat@gmail.com


In [10]:
type(config_data.name)

str

ensure_annotation


In [11]:
def get_product(x:int, y:int):
    return x*y

In [12]:
get_product(x=2, y="7")

'77'

In [13]:
from ensure import ensure_annotations

@ensure_annotations
def get_product(x:int, y:int):
    return x*y

In [15]:
# get_product(x=8, y='8')

Ingesting data from Gdrive

In [1]:
import gdown

In [2]:
url = "https://drive.google.com/file/d/1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt/view?usp=sharing"

In [3]:
type(url)

str

In [4]:
url.split("/")

['https:',
 '',
 'drive.google.com',
 'file',
 'd',
 '1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt',
 'view?usp=sharing']

In [7]:
file_id = url.split("/")[-2] # unique id 

In [8]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "Chest-CT-Scan-data.zip")

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt&confirm=t&uuid=9f559f6d-2518-4854-9f78-e8b5d283e9c4
To: /home/rajat/MediScan-MLOps-Ready-Deep-Learning-Project/research/Chest-CT-Scan-data.zip
100%|██████████| 49.0M/49.0M [00:04<00:00, 12.1MB/s]


'Chest-CT-Scan-data.zip'

In [9]:
import os

In [10]:
os.chdir('../')

In [11]:
%pwd

'/home/rajat/MediScan-MLOps-Ready-Deep-Learning-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-04 17:01:20,351: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-04 17:01:20,358: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-04 17:01:20,359: INFO: common: created directory at: artifacts]
[2025-10-04 17:01:20,360: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-04 17:01:20,361: INFO: 2501729732: Downloading data from https://drive.google.com/file/d/1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt&confirm=t&uuid=ae41afa5-d19f-40ee-9411-5711ef1d10a9
To: /home/rajat/MediScan-MLOps-Ready-Deep-Learning-Project/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:04<00:00, 11.6MB/s]

[2025-10-04 17:01:28,651: INFO: 2501729732: Downloaded data from https://drive.google.com/file/d/1FVubUcibF5ka6TwOYI9etYZK-OCf3dCt/view?usp=sharing into file artifacts/data_ingestion/data.zip]
